In [1]:
import logging
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 全局变量和配置
base_url = "http://guizhou.zxjxjy.com"

# Headers 配置
get_headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Content-Length': '107',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Cookie': 'Hm_lvt_b21659b538990a950b60a32e93668ae4=1721696256; HMACCOUNT=B11A600C74D21728; edu-s=09c8957c97d0486f9a1c00019fc12c8d; Hm_lpvt_b21659b538990a950b60a32e93668ae4=1721697830',
    'Host': 'guizhou.zxjxjy.com',
    'Origin': 'http://guizhou.zxjxjy.com',
    'Referer': 'http://guizhou.zxjxjy.com/p/classroom/simple?coursewareid=fbb5c5dd51284388abf1ee0f46e91387&courseid=c12ec941a16f47a2aa7d8b869f684cea',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

check_headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Cookie': 'Hm_lvt_b21659b538990a950b60a32e93668ae4=1721696256; HMACCOUNT=B11A600C74D21728; edu-s=09c8957c97d0486f9a1c00019fc12c8d; Hm_lpvt_b21659b538990a950b60a32e93668ae4=1721697830',
    'Host': 'guizhou.zxjxjy.com',
    'Referer': 'http://guizhou.zxjxjy.com/p/classroom/simple?coursewareid=fbb5c5dd51284388abf1ee0f46e91387&courseid=c12ec941a16f47a2aa7d8b869f684cea',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
}

submit_headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Content-Length': '127',
    'Content-Type': 'application/json',
    'Cookie': 'Hm_lvt_b21659b538990a950b60a32e93668ae4=1721696256; HMACCOUNT=B11A600C74D21728; edu-s=09c8957c97d0486f9a1c00019fc12c8d; Hm_lpvt_b21659b538990a950b60a32e93668ae4=1721697830',
    'Host': 'guizhou.zxjxjy.com',
    'Origin': 'http://guizhou.zxjxjy.com',
    'Referer': 'http://guizhou.zxjxjy.com/p/classroom/simple?coursewareid=fbb5c5dd51284388abf1ee0f46e91387&courseid=c12ec941a16f47a2aa7d8b869f684cea',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
}

In [4]:
def get_all_cookies(driver):
    """获取当前页面所有cookie信息"""
    all_cookies = driver.get_cookies()
    return {cookie['name']: cookie['value'] for cookie in all_cookies}

def update_headers(original_headers, new_cookie=None, coursewareid=None, courseid=None):
    """更新headers"""
    updated_headers = original_headers.copy()
    
    if new_cookie:
        if isinstance(new_cookie, dict):
            cookie_string = '; '.join(f"{k}={v}" for k, v in new_cookie.items())
            updated_headers['Cookie'] = cookie_string
        elif isinstance(new_cookie, str):
            updated_headers['Cookie'] = new_cookie
        else:
            raise ValueError("new_cookie must be either a dictionary or a string")
    
    if coursewareid or courseid:
        referer = updated_headers['Referer']
        base_url, params = referer.split('?')
        param_dict = dict(param.split('=') for param in params.split('&'))
        
        if coursewareid:
            param_dict['coursewareid'] = coursewareid
        if courseid:
            param_dict['courseid'] = courseid
        
        new_params = '&'.join(f"{k}={v}" for k, v in param_dict.items())
        updated_headers['Referer'] = f"{base_url}?{new_params}"
    
    return updated_headers

def get_action(courseid, coursewareid, lesson_location, headers):
    """发送GET请求"""
    url = f"{base_url}/p/action/get"
    payload = {
        "courseid": courseid,
        "coursewareid": coursewareid,
        "lessonlocation": lesson_location,
    }
    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        # logger.info(f"GET Action Response: {response.status_code} - {response.text}")
    except requests.RequestException as e:
        logger.error(f"GET Action request failed: {e}")

def check_limit(courseid, coursewareid, headers):
    """检查限制"""
    url = f"{base_url}/play/checklimit?courseid={courseid}&cwid={coursewareid}"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        # logger.info(f"Check Limit Response: {response.status_code} - {response.text}")
    except requests.RequestException as e:
        logger.error(f"Check Limit request failed: {e}")

def submit_process(courseid, coursewareid, lesson_location, headers):
    """提交进度"""
    url = f"{base_url}/p/play/submitProcess"
    payload = {
        "courseId": courseid,
        "coursewareId": coursewareid,
        "lessonLocation": round(lesson_location, 6),
    }
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        logger.info(f"Submit Process Response: {response.status_code} - {response.text}")
    except requests.RequestException as e:
        logger.error(f"Submit Process request failed: {e}")


def parse_duration(duration_str):
    """将时间字符串解析为秒数"""
    hours, minutes, seconds = map(int, duration_str.split(':'))
    return hours * 3600 + minutes * 60 + seconds

def parse_progress(progress_str):
    """将进度字符串解析为浮点数"""
    return float(progress_str.strip('%')) / 100

def extract_course_info(html_content):
    """提取课程信息并解析进度和时长"""
    soup = BeautifulSoup(html_content, 'html.parser')
    divs = soup.find_all('div', class_='xszt_ztl')
    
    course_info_list = []
    
    for div in divs:
        course_info = {}
        
        a_tag = div.find('a', class_=['xszt_time jx_xuex', 'xszt_time ks_xuex'])
        if a_tag:
            href = a_tag.get('href', '')
            course_info['coursewareid'] = href.split('coursewareid=')[1].split('&')[0] if 'coursewareid=' in href else None
            course_info['courseid'] = href.split('courseid=')[1] if 'courseid=' in href else None
            course_info['target'] = a_tag.text.strip()
        
        duration_span = div.find('span', class_='color_d')
        if duration_span:
            course_info['duration'] = parse_duration(duration_span.text.strip())
        
        name_span = div.find('span', class_='xszt_name')
        if name_span:
            course_info['name'] = name_span.get('title', '')
        
        progress_em = div.find('em', class_=lambda x: x and x.startswith('jdt_bar_'))
        if progress_em:
            course_info['progress'] = parse_progress(progress_em.text.strip())
        
        course_info_list.append(course_info)
    
    return course_info_list

def determine_start_time(duration, progress):
    """根据进度决定观看起始时间"""
    watched_time = duration * progress
    start_time = max(0, watched_time - 60)  # 从进度前1分钟或0开始
    return round(start_time, 6)



def play_video_in_new_tab(driver, url, max_retries=3):
    original_window = driver.current_window_handle
    print(f"原始窗口句柄: {original_window}")

    for attempt in range(max_retries):
        try:
            print(f"尝试 {attempt + 1}")
            
            # 使用Selenium的原生方法打开新标签页
            driver.switch_to.new_window('tab')
            new_window = driver.current_window_handle
            print(f"新窗口句柄: {new_window}")

            # 在新标签页中打开指定的URL
            driver.get(url)
            print(f"已导航到目标URL: {driver.current_url}")
            
            # 等待页面加载
            time.sleep(5)
            try:
                # 等待元素出现
                element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '#videoContainer > xg-start'))
                )

                # 获取元素的位置和大小
                location = element.location
                size = element.size

                # 计算元素中心点
                x = location['x'] + size['width'] / 2
                y = location['y'] + size['height'] / 2

                # 使用JavaScript模拟点击
                click_script = f"""
                    var evt = new MouseEvent('click', {{
                        bubbles: true,
                        cancelable: true,
                        view: window,
                        clientX: {x},
                        clientY: {y}
                    }});
                    document.elementFromPoint({x}, {y}).dispatchEvent(evt);
                """
                driver.execute_script(click_script)
                print(f"Successfully clicked element at position ({x}, {y})")
                time.sleep(3)
                driver.execute_script(click_script)
                driver.execute_script(click_script)
                print(f"Successfully clicked element at position ({x}, {y})")
                time.sleep(3)
                break
            except Exception as e:
                print(f"Attempt {attempt + 1} failed. Error: {str(e)}")
        except Exception as e:
            print(f"尝试 {attempt+1} 失败: {str(e)}")
            if attempt == max_retries - 1:
                print("达到最大重试次数，操作失败")
                raise
            time.sleep(2)  # 在重试之前稍作等待
        finally:
            # 关闭新标签页并返回原始窗口
            if driver.current_window_handle != original_window:
                driver.close()
            driver.switch_to.window(original_window)
            print("已返回原始页面")
def watch_video(video, driver):
    """观看视频的主要逻辑"""
    courseid = video["courseid"]
    coursewareid = video["coursewareid"]
    duration = video["duration"]
    progress = video["progress"]
    # 打开课程页面并播放
    course_url = f"http://guizhou.zxjxjy.com/p/classroom/simple?coursewareid={coursewareid}&courseid={courseid}"
    play_video_in_new_tab(driver,course_url)
    start_time = determine_start_time(duration, progress)
    logger.info(f"Starting video: courseid={courseid}, coursewareid={coursewareid}, progress={progress:.2%}, start_time={start_time:.2f}")
    
    get_lesson_location = int(start_time)
    submit_lesson_location = start_time
    last_get_time = time.time()
    last_submit_time = time.time()
    last_check_time = time.time()
    
    while submit_lesson_location < duration:
        current_time = time.time()
        cookies = get_all_cookies(driver)
        
        if current_time - last_get_time >= 15:
            get_lesson_location = min(get_lesson_location + 15, duration)
            get_action(courseid, coursewareid, get_lesson_location, update_headers(get_headers, new_cookie=cookies, coursewareid=coursewareid, courseid=courseid))
            last_get_time = current_time

        if current_time - last_check_time >= 15:
            check_limit(courseid, coursewareid, update_headers(check_headers, new_cookie=cookies, coursewareid=coursewareid, courseid=courseid))
            time.sleep(0.5)
            check_limit(courseid, coursewareid, update_headers(check_headers, new_cookie=cookies, coursewareid=coursewareid, courseid=courseid))
            last_check_time = current_time

        if current_time - last_submit_time >= 30:
            submit_lesson_location = min(submit_lesson_location + 30, duration)
            submit_process(courseid, coursewareid, submit_lesson_location, update_headers(submit_headers, new_cookie=cookies, coursewareid=coursewareid, courseid=courseid))
            last_submit_time = current_time

        time.sleep(1)  # 避免过于频繁的循环

    # 确保最后一次提交是完整的视频长度
    if submit_lesson_location < duration:
        submit_process(courseid, coursewareid, duration, update_headers(submit_headers, new_cookie=cookies, coursewareid=coursewareid, courseid=courseid))
    
    logger.info(f"Video completed: courseid={courseid}, coursewareid={coursewareid}, final_time={duration:.2f}")


In [5]:
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "localhost:9998")
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(60)

In [6]:
courses = extract_course_info(driver.page_source)
courses

[{'coursewareid': '1e77839967774672838cc2db7a23fc90',
  'courseid': '415fa43e2efd4c9ca8b26435e38e13a4',
  'target': '开始学习',
  'duration': 2406,
  'name': '固体废物污染控制与资源化（一）',
  'progress': 0.0},
 {'coursewareid': '0c5a515abe5e477b8209431c6deca61a',
  'courseid': '415fa43e2efd4c9ca8b26435e38e13a4',
  'target': '开始学习',
  'duration': 2459,
  'name': '固体废物污染控制与资源化（二）',
  'progress': 0.0},
 {'coursewareid': 'b71fadfd65a64d62b2a1018b0ab6fc7a',
  'courseid': '415fa43e2efd4c9ca8b26435e38e13a4',
  'target': '开始学习',
  'duration': 2433,
  'name': '固体废物污染控制与资源化（三）',
  'progress': 0.0},
 {'coursewareid': 'acdfce92b05a447e9666fb06376a6b99',
  'courseid': '415fa43e2efd4c9ca8b26435e38e13a4',
  'target': '开始学习',
  'duration': 2625,
  'name': '固体废物污染控制与资源化（四）',
  'progress': 0.0},
 {'coursewareid': '61cc46bd29c24d8dbdd611e565435441',
  'courseid': '415fa43e2efd4c9ca8b26435e38e13a4',
  'target': '开始学习',
  'duration': 1557,
  'name': '生活垃圾焚烧发电技术',
  'progress': 0.0},
 {'coursewareid': 'c1a459245bad4de5be499

In [7]:
try:
    for course in courses:
        if course['progress'] < 1:  # 只观看未完成的视频
            watch_video(course, driver)
        else:
            logger.info(f"Skipping completed video: {course['name']}")
    
except Exception as e:
    logger.error(f"An error occurred: {e}")
finally:
    print("All courses have been processed.")


2024-08-05 17:18:29,903 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:19:00,933 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:19:32,049 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:20:02,238 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:20:33,245 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:21:04,262 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:21:35,347 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:22:06,388 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:22:38,338 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:23:09,405 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:23:40,479 - INFO - Submit Process Response: 200 - {
	"code":0,
	"d

原始窗口句柄: 756C5B354C3158336C03CA6B5FED1533
尝试 1
新窗口句柄: D0478CA0E17E69E9D669337850E3366F
已导航到目标URL: http://guizhou.zxjxjy.com/p/classroom/simple?coursewareid=e5eb9129817c4ad5934225c468fd53ff&courseid=ab228fbde3dd4d0ea9544595d4b88b81
Successfully clicked element at position (403.0, 497.0)
Successfully clicked element at position (403.0, 497.0)


2024-08-05 17:51:07,335 - INFO - Starting video: courseid=ab228fbde3dd4d0ea9544595d4b88b81, coursewareid=e5eb9129817c4ad5934225c468fd53ff, progress=0.00%, start_time=0.00


已返回原始页面


2024-08-05 17:51:38,646 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:52:08,773 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:52:38,120 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:53:08,275 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:53:38,371 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:54:08,489 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:54:38,543 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:55:08,603 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:55:38,688 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:56:09,018 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-05 17:56:39,022 - INFO - Submit Process Response: 200 - {
	"code":0,
	"d

...

原始窗口句柄: 756C5B354C3158336C03CA6B5FED1533
尝试 1
新窗口句柄: 55496FEA029690793D1BA2F64DA584F9
已导航到目标URL: http://guizhou.zxjxjy.com/p/classroom/simple?coursewareid=a763cb0bc1004e7ca89b9c9162af41a0&courseid=b7405ed8d2f14b2ea6c7589c351d3109
Successfully clicked element at position (403.0, 497.0)
Successfully clicked element at position (403.0, 497.0)


2024-08-07 00:52:29,720 - INFO - Starting video: courseid=b7405ed8d2f14b2ea6c7589c351d3109, coursewareid=a763cb0bc1004e7ca89b9c9162af41a0, progress=0.00%, start_time=0.00


已返回原始页面


2024-08-07 00:53:01,717 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-07 00:53:32,715 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-07 00:54:01,922 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-07 00:54:32,833 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-07 00:55:02,924 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-07 00:55:33,873 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-07 00:56:03,941 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-07 00:56:33,999 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-07 00:57:04,150 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-07 00:57:34,176 - INFO - Submit Process Response: 200 - {
	"code":0,
	"data":"ok"
}
2024-08-07 00:58:04,270 - INFO - Submit Process Response: 200 - {
	"code":0,
	"d

All courses have been processed.
